As usual, we'll start with cleaning as I won't ever assume data is 'good to go' just because somebody handed it to me.

As a heads up, I find it interesting their focus on RMSLE, something I honstly haven't heard of until a few minutes ago. Likely, a sign that we'll be seeing some logistic relationships, so a heads up on that.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/train.csv/train.csv')
print(df.shape)
df.head() #Quite a nice number of rows...

(90615, 10)


,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [3]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('.', '')
df.columns

Index(['id', 'sex', 'length', 'diameter', 'height', 'whole_weight',
       'whole_weight1', 'whole_weight2', 'shell_weight', 'rings'],
      dtype='object')

In [4]:
# if len(df['id'].unique()) == df.shape[0]:
#     df = df.drop(['id'], axis=1) #Not adding anything to our model... Hmm, now that I think about it, let's make this better:
# else:
#     print(len(df['id'].unique()))

In [5]:
for i in df.columns:
    if len(df[i].unique()) == df.shape[0]:
        df = df.drop([i], axis=1) #Not adding anything to our model... Hmm, now that I think about it, let's make this better:
        print(f"Just dropped {i} as it wasn't adding any value whatsoever to our model.")
    else:
        print(f"Well, {i} has {len(df[i].unique())} unique fields, so we'll keep it...for now!")

#And, on the topic we see all the unique fields too for everything.... Might have to have a different
#approach for larger datasets, but for now... So, pragmatically sex would be the only categorical variable.

Just dropped id as it wasn't adding any value whatsoever to our model.
Well, sex has 3 unique fields, so we'll keep it...for now!
Well, length has 157 unique fields, so we'll keep it...for now!
Well, diameter has 126 unique fields, so we'll keep it...for now!
Well, height has 90 unique fields, so we'll keep it...for now!
Well, whole_weight has 3175 unique fields, so we'll keep it...for now!
Well, whole_weight1 has 1799 unique fields, so we'll keep it...for now!
Well, whole_weight2 has 979 unique fields, so we'll keep it...for now!
Well, shell_weight has 1129 unique fields, so we'll keep it...for now!
Well, rings has 28 unique fields, so we'll keep it...for now!


In [6]:
df.head()

,sex,length,diameter,height,whole_weight,whole_weight1,whole_weight2,shell_weight,rings
0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [7]:
#Oh, before I get too excited let's check for nulls:
if sum(df.isnull().sum())==0:
    print("Phew, no nulls.")
else:
    print(df.columns[df.isnull().any()]) #Honestly thanks to leplandelavile of Stackoverflow for the code for this.... I knew
    #we could find something elegant like this,a s opposed to looping through each column...

Phew no nulls.


In [8]:
# df.loc[df.shape[0]] = ["F", 0,0,0,0,0,0,0, np.nan]
# df.loc[df.shape[0]-1]

Given that we only have on categorical vaiable, we'll just look at that plainly:

In [9]:
df['sex'].value_counts(normalize=True) #Oh interesting, I is actually the majority, then a decent bit more males than females.
#Would be interesting to see their sampling methods.

sex
I    0.365204
M    0.342405
F    0.292391
Name: proportion, dtype: float64

In [29]:
#Before we look at sex x other stuff, let's examine the other columns:
df.dtypes

sex               object
length           float64
diameter         float64
height           float64
whole_weight     float64
whole_weight1    float64
whole_weight2    float64
shell_weight     float64
rings              int64
dtype: object

In [38]:
for i in df.columns[:4]:
    if df[i].dtypes != 'object':
        print(f"{i}")
        print(df[i].describe())
        print(f"~"*26)

length
count    90615.000000
mean         0.517098
std          0.118217
min          0.075000
25%          0.445000
50%          0.545000
75%          0.600000
max          0.815000
Name: length, dtype: float64
~~~~~~~~~~~~~~~~~~~~~~~~~~
diameter
count    90615.000000
mean         0.401679
std          0.098026
min          0.055000
25%          0.345000
50%          0.425000
75%          0.470000
max          0.650000
Name: diameter, dtype: float64
~~~~~~~~~~~~~~~~~~~~~~~~~~
height
count    90615.000000
mean         0.135464
std          0.038008
min          0.000000
25%          0.110000
50%          0.140000
75%          0.160000
max          1.130000
Name: height, dtype: float64
~~~~~~~~~~~~~~~~~~~~~~~~~~


Length is slightly skewed to the left.

So too with diameter. Now, I did a quick Google search of these creatures I've never even heard of until this competition, and typical pictures show them to have an ovalish shape. So, I'm quite confused. I later looked on the original dataset to find that length is'longest shell measurement' while diameter is somehow 'perpendicular to length'. Regardless, we continue.

On the topic of definitions: The three weights correspond to whole, shucked, and viscera weight. Rings correspond to 1.5 years of additional life.

And, height is also skewed to left. Perhaps all the leftists are a trend we'll see more and more. Perhaps we should comment more on that, but for now:

In [40]:
for i in df.columns[4:]:
    if df[i].dtypes != 'object':
        print(f"{i}")
        print(df[i].describe())
        print(f"~"*26)

whole_weight
count    90615.000000
mean         0.789035
std          0.457671
min          0.002000
25%          0.419000
50%          0.799500
75%          1.067500
max          2.825500
Name: whole_weight, dtype: float64
~~~~~~~~~~~~~~~~~~~~~~~~~~
whole_weight1
count    90615.000000
mean         0.340778
std          0.204428
min          0.001000
25%          0.177500
50%          0.330000
75%          0.463000
max          1.488000
Name: whole_weight1, dtype: float64
~~~~~~~~~~~~~~~~~~~~~~~~~~
whole_weight2
count    90615.000000
mean         0.169422
std          0.100909
min          0.000500
25%          0.086500
50%          0.166000
75%          0.232500
max          0.760000
Name: whole_weight2, dtype: float64
~~~~~~~~~~~~~~~~~~~~~~~~~~
shell_weight
count    90615.000000
mean         0.225898
std          0.130203
min          0.001500
25%          0.120000
50%          0.225000
75%          0.305000
max          1.005000
Name: shell_weight, dtype: float64
~~~~~~~~~~~~~~~~~~~

Hmm, now this is quite surprising, (Whole Weight) 0 is ever so slightly skewed to the left;the trend changes when we get into 1 and is still slightly retained at 2 (recall the much lower standard deviations. Pragmatically, I wouldn't be too surprised as I'd imagine that the most varity would occur with the shells, being as dense as they are.

Shell weight appars approximately normal, not accounting for kurtosis

And, finally, rings are to the right. Being an indication of ag, I am unsurprised even in this different environment to see 'those few old guys' prop up again.

Likely visualization would be the way to go, but eh, I'm a numbers guy... Especially when we'll be doing so many feature engineerings... maybe relevant for log transformations later.

In [41]:
#Now, let's see what changes when we divide them up by category.

focus  = ['sex']
for i in focus:
    for c in df.columns[:4]:
        if c != i:
            print(f"{i} on {c}")
            print(df.groupby(i)[c].describe())
            print(f"~"*26)
#Hmm, I wonder if it might be useful to have by it the baseline of df[c].describe() as well. Oh well.

sex on length
       count      mean       std    min    25%    50%    75%    max
sex                                                                
F    26495.0  0.584330  0.071348  0.230  0.545  0.585  0.630  0.815
I    33093.0  0.414792  0.104592  0.075  0.345  0.425  0.495  0.770
M    31027.0  0.568806  0.084280  0.090  0.530  0.575  0.620  0.815
~~~~~~~~~~~~~~~~~~~~~~~~~~
sex on diameter
       count      mean       std    min    25%    50%    75%    max
sex                                                                
F    26495.0  0.458219  0.059050  0.130  0.425  0.460  0.500  0.635
I    33093.0  0.315766  0.085249  0.055  0.260  0.325  0.380  0.600
M    31027.0  0.445032  0.069665  0.090  0.415  0.450  0.485  0.650
~~~~~~~~~~~~~~~~~~~~~~~~~~
sex on height
       count      mean       std   min    25%    50%    75%   max
sex                                                              
F    26495.0  0.157194  0.027658  0.05  0.140  0.155  0.175  1.13
I    33093.0  0.103129  

I'll only bother to write things that significantly pop out.

And, right off th bat we see Is ae much shorter inlength, diameter, and height than hte others.... Hmm, so yeah, that definitely seems like that will be a significant factor. Of special note on hat font are their heights: While their medians aren't toooooo different (well, 50%...), the max is approximately a sixth of the others... 

Also interesting to see in this species the larger diemsnions associated with females, likely associated with how they breed.

In [42]:
focus  = ['sex']
for i in focus:
    for c in df.columns[4:]:
        if c != i:
            print(f"{i} on {c}")
            print(df.groupby(i)[c].describe())
            print(f"~"*26)

sex on whole_weight
       count      mean       std     min     25%     50%     75%     max
sex                                                                     
F    26495.0  1.051711  0.377650  0.0340  0.8130  1.0070  1.2800  2.8255
I    33093.0  0.396115  0.261157  0.0020  0.1870  0.3565  0.5730  2.7795
M    31027.0  0.983810  0.394574  0.0095  0.7635  0.9555  1.1785  2.8255
~~~~~~~~~~~~~~~~~~~~~~~~~~
sex on whole_weight1
       count      mean       std     min     25%     50%     75%     max
sex                                                                     
F    26495.0  0.449880  0.175309  0.0145  0.3285  0.4305  0.5495  1.2695
I    33093.0  0.173873  0.117814  0.0010  0.0785  0.1575  0.2500  1.1455
M    31027.0  0.425631  0.184193  0.0035  0.3125  0.4070  0.5265  1.4880
~~~~~~~~~~~~~~~~~~~~~~~~~~
sex on whole_weight2
       count      mean       std     min     25%     50%     75%    max
sex                                                                    
F    26495

Unsuprisingly per the earlier columns the lsser weight associated with Is. However, their maxes are quite close. And, e the max of male W2 - likely an outlier given hterest of the data with females weighing more (likelya similar matter re. shell wieght's max male weight).

And, the final piece of th data that I was inclined to assume: Notice the much youngr Is re. rings - per medians 3 rings ie 4.5 years younger! So, naturally given that it would make sense for everything to be lower... Ie I'm assuming a continued growth, at whatever rate (linear, logistic, etc.), as time ie rings go on. And, for whatever reason, that is not occuring with Is, hence their typically lower dimensions across the board.

A final piece of curiousity is the disproportionate amount of the classes: While we certainly have quite a large amount of females, they're approximately 5% less than the next largest group of males...Is this an indication that naturally occuring in nature there aren't enough females for all the males? Do Is take care of that? Given the disproportion, does this lead to one group(s) dying earlier or later? On that note, does it lad to one group(s) living differently to change its dimensions?

In [43]:
df['sex'].value_counts(normalize=True)

sex
I    0.365204
M    0.342405
F    0.292391
Name: proportion, dtype: float64

I might use this notebook to prep some images later, but for now we'll call this here; eh, we hardly used it. Let's go ahead and use it to generate new featur